# Calvinball

This project calculates the records for each MLB team when playing by Calvinball rules: any extra inning game or seven-inning game. This project is based on the [pybaseball](https://pypi.org/project/pybaseball/) library.

## Setup

I had some initial problems installing the `pybaseball` library. I tried both installation methods shown on the link above. In the end, performing the following steps got `pybaseball` up and running.

* Upgrade `pip`
* Install `cython`
* Install `cmake`
* Install `pyarrow`
* Install `pybaseball`

## Imports

In order for this script to work, you need to import the `pandas` library, `schedule_and_record` from `pybaseball`, and the various configurations of teams. Teams are represented by a list of strings.

In [1]:
import pandas as pd
from pybaseball import schedule_and_record
from teams import mlb, al, nl, al_east, al_central, al_west, nl_east, nl_central, nl_west

## Functions

Here are the functions used for the script.

### `process_teams`

This function takes a list of teams. It then does the following things:
* Creates an empty data frame with headers for `Team`, `Wins`, `Losses`, and `Win %`.
* Iterate over each team in the list of teams.
* Fetch the record for the team and rename the `W/L` column to `Rec`. **Note:** I had trouble filter this data frame with a header of `W/L`. I think the `/` caused a problem. I renamed the column to avoid this issue. If anybody has an explanation, I would be interested.
* Create the variable `dh_games` which is all double header games. See the explanation for the `find_dh` function below. 
* Create the variable `cb_dh_games` which is all of the Calvinball double header games. See the explanation for the `find_cb_dh` function below.
* Create the variable `cb_games` which is a data frame that has just information for Calvinball games. See the explanation for the `filter_cb_games` function below.
* Create the variable `cb_wins` which is the number of Calvinball wins (regular and double header). See the explanation for the `count_wins` function below.
* Create the variable `cb_losses` which is the number of Calvinball losses (regular and double header). See the explanation for the `count_losses` function below.
* Create the variable `cb_total` which represents the total number of Calvinball games played. It is calculated by finding the length of the `cb_games` data frame and the length of the `cb_dh_games` data frame.
* Create the `cb_win_percentage` variable which is the number of Calvinball wins divided by total Calvinball games.
* Create the `cb_info` variable wich is a dictionary. The keys in the dictionary correspond to the headers in the `cb_records` data frame.
* Finally, append the `cb_info` dictionary to the `cb_records` data frame. The information stored in the dictionary will be converted to a row in the data frame.

In [2]:
def process_teams(teams):
    cb_records = pd.DataFrame(columns=['Team', 'Wins', 'Losses', 'Win %'])
    for team in teams:
        data = schedule_and_record(2021, team).rename(columns={"W/L" : "Rec"})
        dh_games = find_dh(data)
        cb_dh_games = find_cb_dh(dh_games)
        cb_games = filter_cb_games(data)
        cb_wins = count_wins(cb_games) + count_wins(cb_dh_games)
        cb_losses = count_losses(cb_games) + count_losses(cb_dh_games)
        cb_total = len(cb_games) + len(cb_dh_games)
        cb_win_percentage = cb_wins / cb_total
        cb_info = {'Team' : team, 'Wins' : cb_wins, 'Losses' : cb_losses, 'Win %' : cb_win_percentage}
        cb_records = cb_records.append(cb_info, ignore_index=True)
    return cb_records

### `filter_cb_games`

This method takes a data frame for a team's record so far, and returns a data frame with game information if the innings played are greater than 9 or equal to 7.

In [3]:
def filter_cb_games(game_data):
    cb_games = game_data.query('Inn > 9 or Inn == 7')
    return cb_games

### `count_wins`

This function takes a data frame that has the Calvinball information for a particular team. The `wins` variable is a data frame containing only data for when a team wins a Calvinball game. The function returns the length of the `wins` variable, which is the number of Calvinball wins.

In [4]:
def count_wins(game_data):
    wins = game_data.query('Rec == "W" or Rec == "W-wo"')
    return len(wins)


### `count_losses`

This function takes a data frame that has the Calvinball information for a particular team. The `losses` variable is a data frame containing only data for when a team loses a Calvinball game. The function returns the length of the `losses` variable, which is the number of Calvinball losses.

In [5]:
def count_losses(game_data):
    losses = game_data.query('Rec == "L" or Rec == "L-wo"')
    return len(losses)

### `cb_champs`

This function determines who has the best Calvinball record. The function takes a list of teams and returns a data frame. The `cb_data` variable is a data frame representing the team name, number of Calvinball wins, number of Calvinball losses, and the team's Calvinball winning percentage. The data frame is sorted (`.sort_value(by='Win %')`) in descending order (`ascending=False`) and it removes the index (`to_string(index=False)`).

In [6]:
def cb_champs(teams):
    cb_data = process_teams(teams)
    return cb_data.sort_values(by='Win %', ascending=False).to_string(index=False)

### `cb_losers`

This function determines who has the worst Calvinball record. The function takes a list of teams and returns a data frame. The `cb_data` variable is a data frame representing the team name, number of Calvinball wins, number of Calvinball losses, and the team's Calvinball winning percentage. The data frame is sorted in ascending order (`.sort_value(by='Win %')`) and it removes the index (`to_string(index=False)`).

In [7]:
def cb_losers(teams):
    cb_data = process_teams(teams)
    return cb_data.sort_values(by='Win %').to_string(index=False)

### `find_dh`

This function takes the data frame for all games, and returns a data frame of double header games. Start by creating `dh_indices`, an empty list for the indices for double header games. Iterate over the data frame for all of the games. The `itertuples` method creates a named tuple for each row in the data frame. Double headers are indicated by a `(1)` or a `(2)` in the `Date` column. If either of those substrings are present, append the index to `dh_indices`. The list of indices is used to return a data frame of just double header games.

**Note:** I am sure there is a better way of doing this rather than iterating through the data frame row by row. Using the `duplicated` method does not work because the `(1)` and `(2)` keep the dates from beind duplicates.

In [8]:
def find_dh(game_data):
    dh_indices = []
    for items in game_data.itertuples():
        if '(1)' in items.Date or '(2)' in items.Date:
            dh_indices.append(items.Index)
    return game_data.loc[dh_indices]

### `find_cb_dh`

This function takes a data frame of all of the double header games. It returns a data frame of all the Calvinball double headers that have gone past seven innings.

In [9]:
def find_cb_dh(game_data):
    return game_data.query('Inn > 7')

## Running the Script

To run the script, call either `cb_champs` or `cb_losers` and pass them one of the teams variables (`mlb`, `al`, `al_east`, `al_central`, `al_west`, `nl`, `nl_east`, `nl_central`, or `nl_west`).

In [ ]:
print(cb_champs(mlb))